<a href="https://colab.research.google.com/github/lambhua/NLP-Project/blob/main/AI_Chatbot_project_with_Text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the required Libraries**

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from gtts import gTTS  # for text to speech

**Importing data and extracting features to feed in model**

In [6]:
#importing the dataset
# imported content.json from chatbot folder
#content.json on desktop under construction
with open('content.json') as content:
  data1 = json.load(content,strict=False)

#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})

data = data.sample(frac=1)

#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
data

#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

input_shape = x_train.shape[1]
print(input_shape)

#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = le.classes_.shape[0]
print("output length: ",output_length)

10
number of unique words :  87
output length:  8


**Creating the model and training it**

In [7]:
#creating the model

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x) # return sequences to true because we wants hidden state in output at each time step
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)

#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

#training the model
train = model.fit(x_train,y_train,epochs=200)


Epoch 1/200
3/3 [==============================] - 3s 10ms/step - loss: 2.0804 - accuracy: 0.1053
Epoch 2/200
3/3 [==============================] - 0s 9ms/step - loss: 2.0772 - accuracy: 0.1842
Epoch 3/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0746 - accuracy: 0.1711
Epoch 4/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0718 - accuracy: 0.1711
Epoch 5/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0689 - accuracy: 0.1842
Epoch 6/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0667 - accuracy: 0.1842
Epoch 7/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0636 - accuracy: 0.1842
Epoch 8/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0606 - accuracy: 0.1842
Epoch 9/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0574 - accuracy: 0.1842
Epoch 10/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0533 - accuracy: 0.1842
Epoch 11/200
3/3 [==========

**Text to speech function**

In [8]:
# text to speech function
def text_to_speech(text):
    # Initialize gTTS with the text to convert
    speech = gTTS(text = text, lang='en', slow = False)

    # Save the audio file to a temporary file
    speech_file = 'speech.mp3'
    speech.save(speech_file)


    # Play the audio file
    os.system(speech_file)

**Chatting**

In [ ]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')

  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()

  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  resp=random.choice(responses[response_tag])
  print(resp)
  text_to_speech(resp)

  if response_tag == "goodbye":
    break

You : What is your name
1/1 [==============================] - 0s 439ms/step
My name is anoops spiritual bot ready to help you to solve your problems
You : who are you
1/1 [==============================] - 0s 20ms/step
I am a spiritual  Bot developed by anoop , you can call me anoop's bot
You : what is life
1/1 [==============================] - 0s 20ms/step
Samsara can be translated as  the endless wandering . A few words from the Buddha himself  What do you think  Which is greater, the tears you have shed while transmigrating 
          wandering this long  long time crying  weeping from being joined with what is displeasing , being separated from what is pleasing or the water in the four great oceans
          This is the greater  the tears you have shed while transmigrating & wandering this long, long time crying  weeping from being joined with what is displeasing, being separated from 
          what is pleasing not the water in the four great oceans 
You : what is suffering
1/1 